In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_excel('datasets/products.xlsx')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.duplicated().any()
df.duplicated(subset='id').any()

In [ ]:
df.columns[[3,4,10]]

In [ ]:
for i in df.columns[[3,4,10]]:
    df[i]=df[i].fillna('unknown')

In [ ]:
# df[df.columns[[3,4,10]]] = df[df.columns[[3,4,10]]].fillna('unknown')

# df[ ['brand_model', 'discount', 'warranty'] ]=df[ ['brand_model', 'discount', 'warranty'] ].fillna('unknown')


In [ ]:
df.info()

In [ ]:
df['rating'].unique()

In [ ]:
df['rating']=df['rating'].replace({'five':5, 'four':4}).astype(float).round(1)

In [ ]:
df['rating']=df['rating'].fillna(df['rating'].mean()).round(1)

In [ ]:
df.info()

In [ ]:
df=df.set_index('id')
df

In [ ]:
df.columns[[0,1,2,3,7]]

In [ ]:
for col in df.columns[[0,1,2,3,7]]:
    print('-'*5,col,'-'*50)
    print(df[col].unique(),end='\n\n')

In [ ]:
df['product']=df['product'].str.title()
df['stock']=df['stock'].replace({'YES':1,'yes':1,'Y':1,'no':0,'NO':0})
df

In [ ]:
# df.loc[df['discount'].isin(['HP-H01','JBL_450BT','samsung s12','samsung,s12'])]
df.loc[df['brand_model'].isin(['flat 10%','flat 20%','flat ten','upto 30%','flat 15%'])]



In [ ]:
def swap(row):
    bm=row['brand_model'] # scalar
    d=row['discount'] # scalar_data
    if bm in ['flat 10%','flat 20%','flat ten','upto 30%','flat 15%']:
        return [d,bm]
    else:
        return [bm,d]
    

In [ ]:
brand_dis_col=df.apply(swap,axis=1).astype(str).str.replace('[^A-Za-z0-9 ,.%-]','',regex=True).str.rsplit(',',expand=True,n=1)
brand_dis_col.head()

In [ ]:
df[['brand_model','discount']]=brand_dis_col

In [ ]:
df['brand_model'].unique()

In [ ]:
df['discount'].unique()

In [ ]:
# brand_dis_col=df.apply(swap,axis=1) # series of list
# brand_dis_col=brand_dis_col.astype(str) # explicit type conversion
#-------------------------------------------------------------------------------
# brand_dis_col=brand_dis_col.str.replace('[^A-Za-z0-9 ,.%-]','',regex=True) 
# removes the char which is not in the pattern
#-------------------------------------------------------------------------------
# df[['brand_model','discount']]=brand_dis_col.str.rsplit(',',expand=True,n=1) 
# splits using comma and gives list --> expand=True converts the list elements to columns

In [ ]:
df['brand_model'].unique()

In [ ]:
df['brand_model']=df['brand_model'].replace({'Samsung S-12':'Samsung S12'})

In [ ]:
df['brand_model']=df['brand_model'].str.replace(' |,','-',regex=True,n=1,)
df['brand_model'].unique()

In [ ]:
df[['brand','model']]=df['brand_model'].str.split('-',expand=True).fillna('no model')

In [ ]:
df=df.drop(columns='brand_model')

In [ ]:
df

In [ ]:
df['discount'].unique()

In [ ]:
df['discount']=df['discount'].str.strip()

In [ ]:
df['discount']=df['discount'].replace({'none':'No Discount', 'unknown':'No Discount','0.2':'20%'})
df['discount']=df['discount'].str.replace('ten','10',)
df['discount']=df['discount'].str.replace(' percent','%')

In [ ]:
# df.loc[~df['discount'].str.startswith(('upto','flat','No Discount'),),'discount']
#--------------------or--------------------------------------------------------------
# df.loc[~df['discount'].str.contains('upto|flat|No Discount',regex=True,),'discount']


In [ ]:
dis=df.loc[~df['discount'].str.startswith(('upto','flat','No Discount')),'discount']

res='flat '+ dis

df.loc[~df['discount'].str.startswith(('upto','flat','No Discount')),'discount']=res


In [ ]:
df['discount'].unique()

In [ ]:
df['warranty'].unique()

In [ ]:
df['warranty']=df['warranty'].replace({'OneYr':'12 months','1 year':'12 months'})

In [ ]:
df

In [ ]:
for col in ['brand','model']:
    df[col]=df[col].str.title()

In [ ]:
df['seller_city'].unique()

In [ ]:
df['seller_city']=df['seller_city'].str.title()
df['seller_city']=df['seller_city'].replace({'Hyd':'Hyderabad','Blr':'Bangalore'})

In [ ]:
df['price'].unique()

In [ ]:
df['price']=df['price'].astype(str)

In [ ]:
df['price']=df['price'].replace({'one thousand rs':'1000'})

In [ ]:
df['price']=df['price'].str.replace('rs|Rs.|/-| tax| |,','',regex=True,).str.strip(' +')

In [ ]:
df['price']=df['price'].str.replace('USD','$',regex=True,case=False)
df['price']=df['price'].str.replace('k','000',regex=True,case=False)

In [ ]:
df['price'].unique()

In [ ]:
df['price'].unique()

In [ ]:
def conv(p):
    dol=90
    if '+' in p:
        L=p.split('+')

        if L[0].isdigit():  
            v1=float(L[0])
        elif '$' in L[0]:
            v1=float(L[0].strip('$'))*dol

        if L[1].isdigit():
            v2=float(L[1])
        elif '$' in L[1]:
            v2=float(L[1].strip('$'))*dol

        return v1+v2
    
    elif '$' in p:
        return float(p.strip('$'))*dol
    else :
        return float(p)

In [ ]:
df['price']=df['price'].apply(conv).astype(int)

In [ ]:
df['price'].unique()

In [ ]:
df

In [ ]:
df['no_of plt']=df['Amazon']+df['Flipkart']

In [ ]:
df

In [ ]:
df.columns

In [ ]:
col_names=['product', 'price', 'discount', 'rating', 'stock','seller_city', 'warranty', 'brand', 'model','no_of plt']

In [ ]:
df1=pd.melt(df,id_vars=col_names,value_vars=['Amazon','Flipkart'],var_name='shopping app',value_name='availability')
df1=df1.loc[df1['availability']==1].reset_index(drop=True).drop(columns='availability')

In [ ]:
df1

In [ ]:
df.columns

In [ ]:
# 1.which city is selling more products

df1['seller_city'].value_counts().idxmax()

In [ ]:
# 2. which shopping app has more products?

df1['shopping app'].value_counts().head(1)


In [ ]:
#3.get the occurance of product?

df1['product'].value_counts()

In [ ]:
df1.columns

In [ ]:
#4. which product is available in both amazon and filkart


df.loc[df['no_of plt']==2,'product'].unique()

In [ ]:
df1

In [ ]:
# 5. what type of products have warranty.
df.loc[df['product'] != 'unknown', 'product'].unique()

In [ ]:
# 6. get product wise occurrence of brand.
df.groupby('product')['brand'].value_counts()

In [ ]:
# 7. get the product which has either highest or lowest price.
a=df1['price'].agg(['max','min'])
a

In [ ]:
# df1.loc[df1['price'].isin([80000,100]),'product'].unique()
df1.loc[df1['price'].isin(a.values),'product'].unique()

In [ ]:
# 8. if common product is available , then check who is giving more discount

In [ ]:
df1['discount'].unique()

In [ ]:
# df1['discount'].str.extract(r'(\d+)',)

In [ ]:
# import re
# for i in ['flat 10', 'upto 30%', 'flat 20%', 'flat 15%','flat 10%']:
#         print(i,re.findall('(\d+)|(%)',i)[0][0])

In [ ]:
def dis(row):
    p=row['price']
    d=row['discount']
    
    if 'No Discount' in d:
        return p
    
    n=re.findall('(\d+)',d)[0]
    
    if 'upto' in d:
        
        ded=p*(int(n)/200)
        return p-ded

    elif 'flat' in d:
        if '%' in d:
            
            ded=p*(int(n)/100)
            return p-ded
        else:
            
            return p-int(n)

df1['discounted_price']=df1.apply(dis,axis=1)

In [ ]:
df1

In [ ]:
df1.loc[df1['no_of plt']==2].groupby(['product','shopping app']).agg({'discounted_price':'max'})

In [ ]:
# 11. get the relationship b/w price and rating
# df1[['price','rating']].corr()
df1['price'].corr(df1['rating'])

In [ ]:
#12. get the correlation b/w price and rating
df1[['price','rating']].corr()

In [ ]:
#11. get the relationship b/w price and rating for each shopping app
df1.loc[df1['shopping app']=='Amazon',['price','rating']].corr()

In [ ]:
df1.loc[df1['shopping app']=='Flipkart',['price','rating']].corr()

1. which city is selling more products?
2. which shopping app has more products?
3. get the occurrence of products 
4. which product is available in both Amazon and Flipkart.
5. what type of products have warranty.
6. get product wise occurrence of brand.
7. get the product which has either highest or lowest price.
8. if common product is available , then check who is giving more discount
9. get the sales of product for each shopping app
10. get the distribution of (i) price and (ii) rating
11. get the relationship b/w price and rating
12. get the correlation b/w price and rating
11. get the relationship b/w price and rating for each shopping app
12. get the correlation b/w price and rating for each shopping app